imports

In [237]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
import networkx as nx
from scipy.sparse import coo_matrix
from networkx.algorithms.link_analysis import pagerank
from math import log

read ranking data for 100 user

In [238]:
# Read the data from the file
rating_data = pd.read_csv('rating_data.data', header=None, delimiter='\t').values

create sparse matrix from users ranking 

In [239]:



ranking_matrix = coo_matrix((rating_data[:, 2], (rating_data[:, 0], rating_data[:, 1])), shape=(max(rating_data[:, 0]) + 1, max(rating_data[:, 1]) + 1))


print(ranking_matrix.shape)
print(ranking_matrix.getrow(1))


(101, 1239)
  (0, 272)	3
  (0, 271)	2
  (0, 270)	5
  (0, 269)	5
  (0, 268)	5
  (0, 267)	4
  (0, 266)	1
  (0, 265)	4
  (0, 264)	2
  (0, 263)	1
  (0, 262)	3
  (0, 261)	1
  (0, 260)	1
  (0, 259)	1
  (0, 258)	5
  (0, 257)	4
  (0, 256)	4
  (0, 255)	2
  (0, 254)	1
  (0, 253)	5
  (0, 252)	2
  (0, 251)	4
  (0, 250)	4
  (0, 249)	4
  (0, 248)	4
  :	:
  (0, 25)	4
  (0, 24)	3
  (0, 23)	4
  (0, 22)	4
  (0, 21)	1
  (0, 20)	4
  (0, 19)	5
  (0, 18)	4
  (0, 17)	3
  (0, 16)	5
  (0, 15)	5
  (0, 14)	5
  (0, 13)	5
  (0, 12)	5
  (0, 11)	2
  (0, 10)	3
  (0, 9)	5
  (0, 8)	1
  (0, 7)	4
  (0, 6)	5
  (0, 5)	3
  (0, 4)	3
  (0, 3)	4
  (0, 2)	3
  (0, 1)	5


create users info matrix (user) --> list of pref (sorted by time)

In [240]:
# Initialize dictionaries for temporal and item sequences for each user
temporal_sequences = {}
item_sequences = {}

# Populate temporal and item sequences based on rating data
for i in range(1, max(rating_data[:, 0]) + 1):
    temporal_sequences[i] = []
    item_sequences[i] = []

for i in range(rating_data.shape[0]):
    target_user_id, item_id, _, temporal_value = map(int, rating_data[i, :4])

    temporal_sequences[target_user_id].append(temporal_value)
    item_sequences[target_user_id].append(item_id)

# Sort temporal and item sequences for each user based on temporal values
for target_user_id in temporal_sequences.keys():
    if len(temporal_sequences[target_user_id]) > 1:
        sorted_indices = sorted(range(len(temporal_sequences[target_user_id])),
                                key=lambda k: temporal_sequences[target_user_id][k])

        temporal_sequences[target_user_id] = [temporal_sequences[target_user_id][j] for j in sorted_indices]
        item_sequences[target_user_id] = [item_sequences[target_user_id][j] for j in sorted_indices]

# Create a dictionary to store user information based on sorted item sequences
users_info = {user_id: item_sequences[user_id] for user_id in temporal_sequences}

# Print an example of preference for user 1
print("Example preference for user 1:")
print(users_info[1])


Example preference for user 1:
[168, 172, 165, 156, 196, 166, 187, 14, 250, 127, 117, 181, 109, 1, 246, 248, 257, 50, 253, 249, 262, 224, 93, 124, 19, 137, 123, 146, 7, 235, 15, 260, 264, 24, 245, 126, 237, 25, 13, 121, 251, 236, 240, 118, 130, 47, 65, 190, 31, 114, 39, 28, 52, 238, 183, 69, 199, 11, 161, 95, 60, 179, 83, 64, 98, 22, 135, 202, 163, 26, 89, 8, 214, 182, 48, 160, 175, 99, 192, 128, 180, 185, 143, 68, 55, 204, 56, 96, 81, 151, 79, 70, 212, 23, 84, 184, 197, 191, 94, 134, 145, 207, 186, 97, 188, 159, 85, 36, 144, 17, 174, 252, 105, 148, 108, 147, 243, 220, 106, 122, 104, 107, 247, 120, 45, 268, 267, 259, 261, 263, 10, 150, 234, 92, 71, 42, 176, 91, 193, 217, 177, 216, 194, 73, 59, 133, 41, 195, 218, 170, 213, 157, 223, 227, 27, 80, 231, 67, 200, 119, 4, 215, 164, 2, 206, 77, 53, 136, 40, 46, 153, 269, 254, 115, 173, 211, 229, 155, 203, 62, 90, 219, 167, 61, 230, 162, 35, 265, 112, 57, 49, 30, 233, 131, 82, 152, 141, 72, 33, 158, 198, 113, 225, 21, 149, 88, 103, 239, 110, 1

read train data (profile) for my 100 users

In [241]:

train_address = "train_profile.csv"  
train = pd.read_csv(train_address)
train = train.iloc[:, 1:]

def convert_to_numeric(row):
    return pd.to_numeric(row, errors='coerce')

train_data = train.apply(convert_to_numeric, axis=1).values.tolist()


print(train_data)

[[9, 171, 137, 108, 226, 180, 264, 187, 207, 186], [279, 274, 281, 272, 314, 25, 308, 10, 313, 257], [272, 335, 330, 299, 300, 331, 268, 317, 344, 355], [271, 356, 329, 328, 354, 359, 324, 327, 358, 260], [168, 418, 430, 239, 121, 374, 176, 377, 409, 194], [476, 59, 89, 286, 470, 472, 539, 268, 186, 274], [106, 29, 554, 446, 238, 515, 542, 592, 192, 620], [172, 190, 89, 510, 294, 228, 176, 689, 127, 685], [286, 50, 7, 487, 385, 201, 340, 371, 507, 690], [69, 498, 696, 505, 497, 697, 333, 100, 463, 85], [526, 455, 173, 399, 383, 204, 725, 258, 216, 434], [471, 204, 708, 392, 97, 416, 753, 133, 282, 754], [200, 879, 914, 292, 49, 416, 457, 396, 775, 404], [195, 663, 525, 283, 430, 313, 524, 922, 530, 265], [282, 15, 845, 20, 620, 125, 289, 411, 864, 249], [174, 943, 583, 58, 509, 76, 194, 602, 443, 812], [269, 243, 151, 222, 237, 111, 471, 126, 13, 150], [479, 168, 152, 125, 127, 95, 52, 83, 966, 187], [4, 887, 655, 435, 153, 258, 294, 288, 211, 201], [496, 934, 323, 1, 181, 423, 274, 98

create sparse matrix from our train data

In [242]:

rmat_csr = ranking_matrix.tocsr()
trm = coo_matrix((ranking_matrix.shape[0]+1, ranking_matrix.shape[1]+1), dtype=ranking_matrix.dtype)
train_matrix = trm.tocsr()

for i in range(ranking_matrix.shape[0]):
    train_matrix[i, train_data[i-1]] = rmat_csr[i, train_data[i-1]]


# train_matrix=train_matrix.tocoo()
print(train_matrix[1])


  (0, 9)	5
  (0, 108)	5
  (0, 137)	5
  (0, 171)	5
  (0, 180)	3
  (0, 186)	4
  (0, 187)	4
  (0, 207)	5
  (0, 226)	3
  (0, 264)	2


c:\Users\ASUS\anaconda3\Lib\site-packages\scipy\sparse\_index.py:137: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


create list of users that we want to test

In [243]:
testSet =  np.arange(1, ranking_matrix.shape[0])
print(testSet)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]


Split user ratings based on specified target ratings.

    Parameters:
    - user_ratings_matrix (csr_matrix): Sparse user ratings matrix.
    - user_id (int): User ID for whom ratings are split.
    - target_ratings (list): List of target ratings.

    Returns:
    - split_indices (list of arrays): List of arrays containing indices of ratings
      corresponding to each unique target rating for the given user.

In [244]:


def split_user_ratings(user_ratings_matrix, user_id, target_ratings):
    # Extract unique sorted target ratings for the specified user
    unique_target_ratings = np.sort(np.unique(user_ratings_matrix.getrow(user_id).toarray()[0, target_ratings]))
    # Find indices of each unique target rating in the user's ratings
    rating_indices = [np.where(user_ratings_matrix.getrow(user_id).toarray()[0, :] == rating)[0] for rating in unique_target_ratings]

    # Intersect indices with the target ratings indices
    split_indices = [np.intersect1d(indices, target_ratings) for indices in rating_indices]

    return split_indices


Hash function for creating unique identifiers for pairs of indices.

    Parameters:
    - index1 (int): First pref.
    - index2 (int): Second pref.
    - num_rows (int): Number of rows in the rating matrix.
    - num_cols (int): Number of columns in the rating matrix.

    Returns:
    - hash_value (int): Hashed identifier.

In [245]:
def hash_signed(index1, index2, num_rows, num_cols):
    n = num_cols
    i = min(index1, index2)
    
    # Calculate the hashed identifier
    hash_value = num_rows + (((n - 1) * n) // 2) - (((n - i + 1) * (n - i)) // 2) + max(index1, index2) - i
    
    return hash_value


 Create a signed t-list for two sets of ratings.

    Parameters:
    - user_id (int): User ID.
    - ratings_set_1 (list): List of indices for the first set of ratings.
    - ratings_set_2 (list): List of indices for the second set of ratings.
    - rmat (np.ndarray): User-item rating matrix.

    Returns:
    - t_list (np.ndarray): Signed t-list representation.

In [246]:
import numpy as np

def calculate_signed_train_list(user_id, ratings_set_1, ratings_set_2, rmat):
    num_combinations = len(ratings_set_1) * len(ratings_set_2)
    t_list = np.zeros((num_combinations, 3))
    count = 1

    for index_1 in range(len(ratings_set_1)):
        for index_2 in range(len(ratings_set_2)):
            if ratings_set_2[index_2] < ratings_set_1[index_1]:
                t_list[count - 1, :] = [user_id,
                                        hash_signed(ratings_set_2[index_2], ratings_set_1[index_1], rmat.shape[0]-1, rmat.shape[1]-1),
                                        -1]
            else:
                t_list[count - 1, :] = [user_id,
                                        hash_signed(ratings_set_2[index_2], ratings_set_1[index_1], rmat.shape[0]-1, rmat.shape[1]-1),
                                        1]
            count += 1

    return t_list


 Construct a signed graph based on user ratings.

    Parameters:
    - rmat (np.ndarray): User-item rating matrix.
    - tr (list of lists): List of lists containing target ratings for each user.

    Returns:
    - signed_graph (np.ndarray): Signed graph representation based on user ratings.

In [247]:
import numpy as np

def create_signed_graph(rmat, tr):
    signed_graph = np.zeros((3,))  # Assuming the matrix is initialized with zeros

    for user_id in range(1, rmat.shape[0]):
        s = split_user_ratings(rmat, user_id, tr[user_id-1])
        count = 0
        temp_graph = np.zeros((3,))

        if len(s) > 1:
            for j in range(len(s) - 1):
                for k in range(j + 1, len(s)):
                    if len(s[j]) > 0 and len(s[k]) > 0:
                        temp_t = calculate_signed_train_list(user_id, s[j], s[k], rmat)
                        temp_graph = np.vstack((temp_graph, temp_t))
                        count += 1

        if count > 0:
            signed_graph = np.vstack((signed_graph, temp_graph[1:, ]))

    signed_graph = signed_graph[1:, :]
    return signed_graph


In [248]:

edge_list = create_signed_graph(ranking_matrix, train_data)


In [249]:
print(edge_list)

[[ 1.00000e+00  2.05676e+05 -1.00000e+00]
 [ 1.00000e+00  2.53263e+05 -1.00000e+00]
 [ 1.00000e+00  2.12003e+05 -1.00000e+00]
 ...
 [ 1.00000e+02  3.49839e+05  1.00000e+00]
 [ 1.00000e+02  3.50049e+05  1.00000e+00]
 [ 1.00000e+02  3.50384e+05  1.00000e+00]]


SiBreNet is a bipartite graph capturing user preferences with positive edges denoting agreement and negative edges indicating disagreement. Users are connected to preference nodes, forming a structured representation of opinions on ranked preferences. The algorithm organizes preferences alphabetically, creating nodes for each user and preference pair. SiBreNet facilitates the analysis of nuanced user preferences, distinguishing between agreements, disagreements, and unknown judgments.

In [250]:

def convert_edgelist_to_graph(l):
    g = nx.Graph()
    for edge in l[:, :2].astype(int):
        g.add_edge(*edge)
    
    weights = {edge: weight for edge, weight in zip(g.edges(), l[:, 2])}
    nx.set_edge_attributes(g, weights, 'weight')

    
    # Print the edges
    for edge in g.edges(data=True):
     print(f"{edge[0]} ==> {edge[1]}, Weight: {edge[2]['weight']}")
    
    return g

g=convert_edgelist_to_graph(edge_list)

1 ==> 205676, Weight: -1.0
1 ==> 253263, Weight: -1.0
1 ==> 212003, Weight: -1.0
1 ==> 213054, Weight: -1.0
1 ==> 10223, Weight: -1.0
1 ==> 126944, Weight: -1.0
1 ==> 159279, Weight: -1.0
1 ==> 196118, Weight: -1.0
1 ==> 233864, Weight: -1.0
1 ==> 205598, Weight: 1.0
1 ==> 205599, Weight: 1.0
1 ==> 211965, Weight: -1.0
1 ==> 213016, Weight: -1.0
1 ==> 10139, Weight: -1.0
1 ==> 126860, Weight: -1.0
1 ==> 159195, Weight: -1.0
1 ==> 196034, Weight: -1.0
1 ==> 205619, Weight: 1.0
1 ==> 10185, Weight: -1.0
1 ==> 126906, Weight: -1.0
1 ==> 159241, Weight: -1.0
1 ==> 196080, Weight: -1.0
1 ==> 233826, Weight: -1.0
1 ==> 10145, Weight: -1.0
1 ==> 126866, Weight: -1.0
1 ==> 159201, Weight: -1.0
1 ==> 196040, Weight: -1.0
1 ==> 211946, Weight: 1.0
1 ==> 10146, Weight: -1.0
1 ==> 126867, Weight: -1.0
1 ==> 159202, Weight: -1.0
1 ==> 196041, Weight: -1.0
1 ==> 212997, Weight: 1.0
159279 ==> 76, Weight: -1.0
2 ==> 11501, Weight: -1.0
2 ==> 300713, Weight: -1.0
2 ==> 307433, Weight: -1.0
2 ==> 33289

normalize nodes

In [251]:

original_weights = nx.get_edge_attributes(g, 'weight')


original_order = sorted(g.nodes())
mapping = {old_node: new_node for new_node, old_node in enumerate(original_order, start=1)}
g = nx.relabel_nodes(g, mapping)


new_weights = {(mapping[edge[0]], mapping[edge[1]]): weight for edge, weight in original_weights.items()}
nx.set_edge_attributes(g, new_weights, 'weight')


for edge in g.edges(data=True):
    print(f"{edge[0]} ==> {edge[1]}, Weight: {edge[2]['weight']}")


1 ==> 1448, Weight: -1.0
1 ==> 1744, Weight: -1.0
1 ==> 1497, Weight: -1.0
1 ==> 1509, Weight: -1.0
1 ==> 227, Weight: -1.0
1 ==> 856, Weight: -1.0
1 ==> 1147, Weight: -1.0
1 ==> 1340, Weight: -1.0
1 ==> 1659, Weight: -1.0
1 ==> 1442, Weight: 1.0
1 ==> 1443, Weight: 1.0
1 ==> 1496, Weight: -1.0
1 ==> 1508, Weight: -1.0
1 ==> 223, Weight: -1.0
1 ==> 852, Weight: -1.0
1 ==> 1140, Weight: -1.0
1 ==> 1336, Weight: -1.0
1 ==> 1447, Weight: 1.0
1 ==> 226, Weight: -1.0
1 ==> 855, Weight: -1.0
1 ==> 1144, Weight: -1.0
1 ==> 1339, Weight: -1.0
1 ==> 1658, Weight: -1.0
1 ==> 224, Weight: -1.0
1 ==> 853, Weight: -1.0
1 ==> 1142, Weight: -1.0
1 ==> 1337, Weight: -1.0
1 ==> 1495, Weight: 1.0
1 ==> 225, Weight: -1.0
1 ==> 854, Weight: -1.0
1 ==> 1143, Weight: -1.0
1 ==> 1338, Weight: -1.0
1 ==> 1507, Weight: 1.0
1147 ==> 76, Weight: -1.0
2 ==> 246, Weight: -1.0
2 ==> 2189, Weight: -1.0
2 ==> 2248, Weight: -1.0
2 ==> 2494, Weight: -1.0
2 ==> 333, Weight: -1.0
2 ==> 1981, Weight: -1.0
2 ==> 2245, Weig

SiBreNet 

In [252]:
print(g)

Graph with 3282 nodes and 3252 edges


 Compute positive and negative SRank values for nodes in the SiBreNet graph.

    Parameters:
    - graph (nx.Graph): The SiBreNet graph.
    - query_node (int): The query node for personalized random walk.
    - damping (float): Damping factor for the teleport probability during the random walk.
    - max_iterations (int): Maximum number of iterations.

    Returns:
    - positive_srank (np.ndarray): Positive SRank values for each node.
    - negative_srank (np.ndarray): Negative SRank values for each node.

In [253]:
import networkx as nx
import numpy as np

def srank(graph, query_node, damping=0.85, max_iterations=10):


    # Step 1: Initialization
    num_nodes = len(graph.nodes) + 1
    transition_matrix_positive = np.zeros((num_nodes, num_nodes))
    transition_matrix_negative = np.zeros((num_nodes, num_nodes))

    # Build transition matrices for positive and negative edges
    for edge in graph.edges(data=True):
        i, j, weight = edge

        if weight['weight'] == 1:
            transition_matrix_positive[i, j] = 1 / graph.degree(j)
            transition_matrix_positive[j, i] = 1 / graph.degree(i)
        elif weight['weight'] == -1:
            transition_matrix_negative[i, j] = 1 / graph.degree(j)
            transition_matrix_negative[j, i] = 1 / graph.degree(i)

   
    # Initialize personalization vector
    personalization_vector = np.zeros(num_nodes)
    personalization_vector[query_node] = 1
    personalization_vector = personalization_vector.reshape((num_nodes, 1))

    # Step 2: Initialize SRank values
    positive_srank = np.ones(num_nodes)
    positive_srank[0] = 0
    negative_srank = np.ones(num_nodes)
    negative_srank[0] = 0
    total_sum = np.sum(positive_srank) + np.sum(negative_srank)

    # Step 3: Normalize SRank values
    positive_srank /= total_sum
    negative_srank /= total_sum
    positive_srank = positive_srank.reshape((num_nodes, 1))
    negative_srank = negative_srank.reshape((num_nodes, 1))

    # Step 4: Iteration
    for i in range(max_iterations):
        # Update positive SRank values
        positive_srank_new = damping * (
            np.dot(transition_matrix_positive, positive_srank)
            + np.dot(transition_matrix_negative, negative_srank)
        ) + (1 - damping) * personalization_vector

        # Update negative SRank values
        negative_srank_new = damping * (
            np.dot(transition_matrix_negative, positive_srank)
            + np.dot(transition_matrix_positive, negative_srank)
        )

        


        # Update SRank values
        positive_srank = positive_srank_new
        negative_srank = negative_srank_new
     

    # Step 5: Compute overall SRank for each node
    return positive_srank, negative_srank
# query_node = 1
# s_rank_result = srank(g, 1)
# # print("SRank values:")
# # print("pos srank:\n",s_rank_result[0])
# # print("neg srank:\n",s_rank_result[1])

example of srank for user 1 

In [254]:
query_node = 1
s_rank_result = srank(g, 1)
print("SRank values:")
print("pos srank:\n",s_rank_result[0])
print("neg srank:\n",s_rank_result[1])

SRank values:
pos srank:
 [[0.00000000e+00]
 [4.25300680e-01]
 [2.99930537e-05]
 ...
 [2.95270691e-05]
 [2.94617988e-05]
 [2.95587493e-05]]
neg srank:
 [[0.00000000e+00]
 [2.98455135e-05]
 [2.99930537e-05]
 ...
 [2.95270691e-05]
 [2.94617988e-05]
 [2.95587493e-05]]


find srank for all user in test set

Calculate SRank values for test users based on the given graph.

    Parameters:
    - graph (nx.Graph): The input graph.
    - test_set (np.ndarray): Array containing test user nodes.

    Returns:
    - sim (np.ndarray): Matrix containing positive and negative SRank values for test users.

Calculate SRank values for test users based on the SiBreNet graph.

    Parameters:
    - graph (nx.Graph): The input graph.
    - test_set (np.ndarray): Array containing test user nodes.

    Returns:
    - sim (np.ndarray): Matrix containing positive and negative SRank values for test users.

In [255]:
def calculate_srank_for_test_users(graph, test_set):
 
    num_test_users = len(test_set)
    sim = np.zeros((num_test_users, 2 * num_test_users))

    for i, test_user in enumerate(test_set, start=1):
        # Calculate SRank values for the current test user
        srank_values = srank(graph, test_user)
        srank_values = (srank_values[0][test_set].flatten(), srank_values[1][test_set].flatten())

        # Populate the similarity matrix
        sim[i-1, :] = np.concatenate((srank_values[0], srank_values[1]))
        sim[i-1, i-1] = 0
        sim[i-1, i-1 + num_test_users] = 0

    return sim

# Example usage
similarity_matrix = calculate_srank_for_test_users(graph=g, test_set=testSet)


In [ ]:

print(similarity_matrix[0,:])

[0.00000000e+00 2.99930537e-05 3.01802476e-05 2.97610951e-05
 3.06802368e-05 3.03236062e-05 2.99930537e-05 2.99930537e-05
 2.91523776e-05 2.99930537e-05 3.01228219e-05 2.99930537e-05
 2.99930537e-05 2.99930537e-05 3.05363727e-05 2.99893485e-05
 3.06183248e-05 2.99930537e-05 2.96337928e-05 3.08358532e-05
 2.99930537e-05 2.99930537e-05 2.99930537e-05 2.99755871e-05
 2.77210684e-05 3.11928219e-05 2.99046851e-05 2.99930537e-05
 2.98025538e-05 2.89967691e-05 2.99874692e-05 3.16320853e-05
 2.77135431e-05 2.96397506e-05 3.00460845e-05 2.96450514e-05
 2.96625012e-05 2.99930537e-05 3.06643725e-05 3.05340021e-05
 2.97491030e-05 2.99930537e-05 2.99930537e-05 1.28591137e-04
 2.89900847e-05 3.00593088e-05 2.99548970e-05 2.99930537e-05
 2.99930537e-05 2.99930537e-05 3.19559465e-05 2.99930537e-05
 2.88764033e-05 2.99930537e-05 3.15211024e-05 2.99930537e-05
 3.03211785e-05 2.99930537e-05 2.99930537e-05 3.07150157e-05
 2.99133581e-05 2.99930537e-05 2.99930537e-05 2.98143864e-05
 2.92082654e-05 2.856081

Calculate the similarity matrix based on the given SRank similarity matrix.

    Parameters:
    - similarity_matrix (np.ndarray): SRank similarity matrix.

    Returns:
    - Sim (np.ndarray): Calculated similarity matrix.

In [ ]:
def calculate_similarity_matrix(similarity_matrix):
 

    num_test_users = similarity_matrix.shape[0]
    index = num_test_users
    Sim = np.zeros((num_test_users, num_test_users))
    # Extract positive and negative SRank values from the similarity matrix
    pos_srank_values = similarity_matrix[:, 0:index]
   
    neg_srank_values = similarity_matrix[:, index:]
    # Calculate the similarity matrix using the given formula
    Sim[:, 0:index] = (pos_srank_values - neg_srank_values) / (neg_srank_values + pos_srank_values)

    # Set diagonal elements to zero
    np.fill_diagonal(Sim, 0)

    return Sim

# Example usage
resulting_similarity_matrix = calculate_similarity_matrix(similarity_matrix)

 
print(resulting_similarity_matrix[0, :])


[ 0.00000000e+00  0.00000000e+00 -3.97583333e-05  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2.50897207e-05  0.00000000e+00 -1.05379898e-02  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -2.37439375e-02  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -3.82105812e-05  0.00000000e+00  3.66882393e-05  0.00000000e+00
  0.00000000e+00 -6.30482605e-01  4.00171173e-05  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -3.91306588e-05  1.31183695e-02
  3.68801188e-05  0.00000000e+00  0.00000000e+00  6.15590731e-01
  3.78457505e-05  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.01584363e-02  0.00000000e+00
  2.72779885e-05  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.29544419e-02
  0.00000000e+00  0.00000

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21036\2419880936.py:12: RuntimeWarning: invalid value encountered in divide
  Sim[:, 0:index] = (pos_srank_values - neg_srank_values) / (neg_srank_values + pos_srank_values)


Write the Similarity to a CSV file

In [ ]:
Sim_df = pd.DataFrame(resulting_similarity_matrix)


simdirec = "Similarity.csv"

# Write the DataFrame to a CSV file
Sim_df.to_csv(simdirec, index=False)

 Estimate the preference matrix for a target user based on their neighbors.

    Parameters:
    - similarity_matrix (numpy.ndarray): Matrix of user similarities.
    - ranking_matrix (numpy.ndarray): Matrix representing user rankings.
    - train_matrix (numpy.ndarray): Matrix representing training data.
    - target_user_id (int): ID of the target user.
    - train_data (list): List of training data.
    - top_k (int): Number of top similar users to consider.

    Returns:
    - preference_matrix (numpy.ndarray): Estimated preference matrix for the target user.


In [ ]:

def estimate_preference_matrix(similarity_matrix, ranking_matrix, train_matrix, target_user_id, train_data, top_k):
    # Sort the similarity values in descending order
    sorted_similarities = np.argsort(similarity_matrix[target_user_id, :])[::-1]
    
    # Extract the top-k similar users and their similarity values
    sim_values = similarity_matrix[target_user_id, sorted_similarities[:top_k]].astype(float)
    
    # Initialize preference_matrix and similarity_impact matrices
    preference_matrix = np.zeros((ranking_matrix.shape[1], ranking_matrix.shape[1]))
    similarity_impact = np.zeros((ranking_matrix.shape[1], ranking_matrix.shape[1]))

    # Loop over the top-k similar users
    for k in range(len(sim_values)):
        selected_user_id = sorted_similarities[k] + 1
        
        # Split user ratings based on train_data
        trs = split_user_ratings(ranking_matrix, selected_user_id, train_data[sorted_similarities[k]])
        
        # Check if similarity value is not NaN and there are at least 2 split ratings
        if not np.isnan(sim_values[k]) and len(trs) >= 2:
            # Loop through pairs of split ratings
            for i in range(len(trs) - 1):
                for j in range(i + 1, len(trs)):
                    # Loop through individual ratings in the pairs
                    for a in range(len(trs[i])):
                        for b in range(len(trs[j])):
                            # Update preference_matrix and similarity_impact
                            preference_matrix[trs[i][a], trs[j][b]] += (
                                sim_values[k] * (train_matrix[selected_user_id, trs[j][b]] - train_matrix[selected_user_id, trs[i][a]])
                            )
                            preference_matrix[trs[j][b], trs[i][a]] -= (
                                sim_values[k] * (train_matrix[selected_user_id, trs[j][b]] - train_matrix[selected_user_id, trs[i][a]])
                            )
                            similarity_impact[trs[i][a], trs[j][b]] += abs(sim_values[k])
                            similarity_impact[trs[j][b], trs[i][a]] = similarity_impact[trs[i][a], trs[j][b]]

    # Find non-zero indices in similarity_impact matrix
    not_zero = np.where(similarity_impact != 0)


    # Formula: ψˆu_i,j = Σ_{v∈neighborhood(u)} (Su(v) * ψv_i,j) / Σ_{v∈neighborhood(u)} |Su(v)|
    # Update preference_matrix using exponential transformation and normalization
    preference_matrix[not_zero] = preference_matrix[not_zero] / similarity_impact[not_zero]

    # Formula: ϕ(i, j) = e^{ψˆu_i,j} / Σ_{k∈I} e^{ψˆu_i,k}
    preference_matrix[not_zero] = np.exp(preference_matrix[not_zero])

    return preference_matrix


Function to infer the total ranking using PageRank

In [ ]:

def infer_total_ranking(preference_matrix):
    # Create a directed graph from the preference_matrix
    graph = nx.DiGraph(preference_matrix)
    
    # Use PageRank to infer total ranking
    total_ranking = pagerank(graph)
    
    return total_ranking




calculate preference_matrix  for user "1" 

In [ ]:

top_k = 10
target_user_id=1
preference_matrix = estimate_preference_matrix(resulting_similarity_matrix, ranking_matrix, train_matrix, target_user_id-1, train_data, 10)
pref_ranking_data = infer_total_ranking(preference_matrix)
print(pref_ranking_data)

{0: 0.0005903086609830849, 1: 0.0005903086609830849, 2: 0.0005903086609830849, 3: 0.0005903086609830849, 4: 0.0005903086609830849, 5: 0.0005903086609830849, 6: 0.0005903086609830849, 7: 0.016108223930681228, 8: 0.0005903086609830849, 9: 0.007027067375543983, 10: 0.0005903086609830849, 11: 0.0005903086609830849, 12: 0.0005903086609830849, 13: 0.0005903086609830849, 14: 0.0005903086609830849, 15: 0.0005903086609830849, 16: 0.0005903086609830849, 17: 0.0005903086609830849, 18: 0.0005903086609830849, 19: 0.0005903086609830849, 20: 0.0005903086609830849, 21: 0.002354673942754664, 22: 0.0005903086609830849, 23: 0.0005903086609830849, 24: 0.0005903086609830849, 25: 0.001110837473599395, 26: 0.0005903086609830849, 27: 0.0005903086609830849, 28: 0.0005903086609830849, 29: 0.0005903086609830849, 30: 0.0005903086609830849, 31: 0.0017847636722764777, 32: 0.004626115781350633, 33: 0.0005903086609830849, 34: 0.0005903086609830849, 35: 0.0005903086609830849, 36: 0.0005903086609830849, 37: 0.000590308

find top 10 pref  for user 1

In [ ]:


# Sort the dictionary by values in descending order
sorted_pref_result = dict(sorted(pref_ranking_data.items(), key=lambda item: item[1], reverse=True))

# Take the top-K items
top_k_elements = list(sorted_pref_result.items())[:top_k]

# Print or use the values
print("Top-K elements for user 1:", top_k_elements)


Top-K elements for user 1: [(276, 0.021239421134780074), (288, 0.017079380364101438), (7, 0.016108223930681228), (64, 0.01066244125718063), (50, 0.009919368159916727), (178, 0.009919368159916727), (181, 0.007951155389618329), (135, 0.007932577623674589), (137, 0.007932577623674589), (83, 0.007085794636080774)]


# Evaluation


Find the positions in the user info that are not present in the train data

In [ ]:


target_positions = [index_user_info for target_value in train_data[0] for index_user_info, user_info_value in enumerate(users_info[testSet[0]]) if user_info_value == target_value]

remaining_positions = [user_info_value for index_user_info, user_info_value in enumerate(users_info[testSet[0]]) if index_user_info not in target_positions]

print("positions in the user info that are not present in the train data:",remaining_positions)


positions in the user info that are not present in the train data: [168, 172, 165, 156, 196, 166, 14, 250, 127, 117, 181, 109, 1, 246, 248, 257, 50, 253, 249, 262, 224, 93, 124, 19, 123, 146, 7, 235, 15, 260, 24, 245, 126, 237, 25, 13, 121, 251, 236, 240, 118, 130, 47, 65, 190, 31, 114, 39, 28, 52, 238, 183, 69, 199, 11, 161, 95, 60, 179, 83, 64, 98, 22, 135, 202, 163, 26, 89, 8, 214, 182, 48, 160, 175, 99, 192, 128, 185, 143, 68, 55, 204, 56, 96, 81, 151, 79, 70, 212, 23, 84, 184, 197, 191, 94, 134, 145, 97, 188, 159, 85, 36, 144, 17, 174, 252, 105, 148, 147, 243, 220, 106, 122, 104, 107, 247, 120, 45, 268, 267, 259, 261, 263, 10, 150, 234, 92, 71, 42, 176, 91, 193, 217, 177, 216, 194, 73, 59, 133, 41, 195, 218, 170, 213, 157, 223, 227, 27, 80, 231, 67, 200, 119, 4, 215, 164, 2, 206, 77, 53, 136, 40, 46, 153, 269, 254, 115, 173, 211, 229, 155, 203, 62, 90, 219, 167, 61, 230, 162, 35, 265, 112, 57, 49, 30, 233, 131, 82, 152, 141, 72, 33, 158, 198, 113, 225, 21, 149, 88, 103, 239, 110, 

Find the positions in the pref_ranking_data that are in the remaining_positions


(Recommend preferences that exist in the ranking data but are not present in our training data)

In [ ]:
pref_ranking_data = [pref_ranking_data[i] for i in remaining_positions]

print(pref_ranking_data)

[0.0005903086609830849, 0.0005903086609830849, 0.0005903086609830849, 0.0005903086609830849, 0.0017847636722764777, 0.0005903086609830849, 0.0005903086609830849, 0.0010037765941650913, 0.0005903086609830849, 0.0005903086609830849, 0.007951155389618329, 0.0005903086609830849, 0.0005903086609830849, 0.0005903086609830849, 0.0005903086609830849, 0.0005903086609830849, 0.009919368159916727, 0.0005903086609830849, 0.0005903086609830849, 0.004626115781350633, 0.0005903086609830849, 0.0005903086609830849, 0.0005903086609830849, 0.0005903086609830849, 0.0005903086609830849, 0.0005903086609830849, 0.016108223930681228, 0.0005903086609830849, 0.0005903086609830849, 0.0005903086609830849, 0.0005903086609830849, 0.0005903086609830849, 0.0005903086609830849, 0.0005903086609830849, 0.001110837473599395, 0.0005903086609830849, 0.0056785688164170565, 0.0005903086609830849, 0.0005903086609830849, 0.0005903086609830849, 0.003513256320007622, 0.0005903086609830849, 0.0005903086609830849, 0.00059030866098

find ranking array for user 1 in the remaining_positions(position that are not in the train data)

In [ ]:

ranking_data_for_user_1=ranking_matrix.toarray()[1,remaining_positions]
print("ranking array for user 1")
print(ranking_data_for_user_1)

ranking array for user 1
[5 5 5 4 5 5 5 4 5 3 5 5 5 5 4 4 5 5 4 3 5 5 5 5 4 4 4 5 5 1 3 2 2 2 4 5 4
 4 4 3 3 3 4 4 5 3 5 4 4 4 4 5 3 4 2 4 4 5 3 3 5 4 4 4 5 4 3 5 1 4 4 5 4 5
 3 4 4 4 1 4 5 5 4 5 5 4 4 3 4 4 4 4 5 5 2 4 2 3 3 3 3 2 4 3 5 2 2 2 3 1 3
 4 3 1 4 1 1 5 5 4 1 1 1 3 5 4 3 3 5 5 5 4 3 5 5 4 3 5 4 2 5 3 5 2 4 5 4 2
 4 1 3 3 5 3 3 3 3 4 4 3 3 3 4 3 5 1 5 5 3 4 2 4 3 4 1 2 4 4 4 1 4 1 5 3 3
 2 1 5 5 3 4 4 3 5 5 2 1 2 4 1 4 1 2 2 4 1 4 4 3 3 5 3 5 3 4 4 1 2 4 3 1 4
 4 1 3 2 3 4 2 4 3 5 5 5 5 5 5 5 5 5 4 5 1 2 3 2 4 5 5 5 2 4 5 4 5 3 5 5 4
 3 1 2]


 Calculate the weighted sum using the Discounted Cumulative Gain (DCG) formula


 
 DCG formula: Σ((2^raking_data - 1) / log(i + 1))

In [ ]:


def DCG(ranking_data):
    # Generate positions from 1 to the length of the ranking_data
    i = np.arange(1, len(ranking_data) + 1)
    
    # Formula: Σ((2^raking_data - 1) / log(i + 1))
    dcg = np.sum((2 ** ranking_data - 1) / np.log2(i + 1))
    
    return dcg


find Discounted Cumulative Gain (DCG) and Ideal Discounted Cumulative Gain (IDCG) and NDCG= IDCG/DCG

In [ ]:

def NDCG(recommended_ranking, scores, top_k):
    recommended_indices = np.argsort(recommended_ranking)[::-1][:top_k]
    recommended_scores = scores[recommended_indices]
    
    #Discounted Cumulative Gain (DCG)
    normalized_dcg = DCG(recommended_scores)

    ideal_indices = np.argsort(scores)[::-1][:top_k]
    ideal_scores = scores[ideal_indices]

    #Ideal Discounted Cumulative Gain (IDCG)
    ideal_dcg = DCG(ideal_scores)
    #NDCG= IDCG/DCG
    return normalized_dcg / ideal_dcg




Calculate the NDCG value for user 1

In [ ]:
NDCG_result = NDCG(pref_ranking_data, ranking_data_for_user_1, 10)
print(NDCG_result)

0.6862750183143672
